
#  PARTIE 2 — Statistique bivariée
Analyse des relations entre deux variables à l’aide de tableaux synthétiques et graphiques.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set(style="whitegrid")
except:
    sns = None

df = pd.read_csv("ecommerce_dataset.csv")
df.head()



## 🧭 Récap des méthodes bivariées 

| Relation | Exemple | Indicateurs | Graphiques |
|---------|---------|-------------|------------|
| QUANTI ↔ QUANTI | revenu_mensuel ↔ montant_panier | covariance, corrélation | scatter plot |
| QUALI ↔ QUANTI (2 modalités) | sexe ↔ montant_panier | moyennes par groupe | boxplot, violin plot |
| QUALI ↔ QUANTI (3+ modalités) | segment_age ↔ montant_panier | moyennes par groupe | boxplot, barplot |
| QUALI ↔ QUALI | sexe ↔ retour_produit | table de contingence | countplot, heatmap |




#  CAS 1 — QUANTI ↔ QUANTI
**Exemple : revenu_mensuel ↔ montant_panier**



###  Tableau synthétique

| Élément | Contenu |
|--------|---------|
| Variables | revenu_mensuel (X), montant_panier (Y) |
| Indicateurs | Covariance, Corrélation |
| Graphique | Scatter plot |
| Objectif | Voir la force et le sens de la relation |


In [ ]:
import numpy as np
from scipy import stats

# Variables quantitatives
x = df["revenu_mensuel"]
y = df["montant_panier"]


plt.figure()
if sns: sns.scatterplot(x=x, y=y, alpha=0.6)
else: plt.scatter(x, y, alpha=0.6)
plt.xlabel("Revenu mensuel")
plt.ylabel("Montant du panier")
plt.title("Scatter plot — revenu vs panier")
plt.show()

In [ ]:



# --- Covariance ---
cov_xy = np.cov(x, y, ddof=1)[0, 1]

# --- Corrélation de Pearson ---
pearson_r, pearson_p = stats.pearsonr(x, y)

# --- Corrélation de Spearman (monotone) ---
spearman_rho, spearman_p = stats.spearmanr(x, y)

# --- Corrélation de Kendall (τ) ---
kendall_tau, kendall_p = stats.kendalltau(x, y)

print(" Covariance :", cov_xy)
print(f" Corrélation de Pearson : r = {pearson_r:.3f} (p = {pearson_p:.3e})")
print(f" Corrélation de Spearman : ρ = {spearman_rho:.3f} (p = {spearman_p:.3e})")
print(f" Corrélation de Kendall : τ = {kendall_tau:.3f} (p = {kendall_p:.3e})")


# Interprétation de la covariance et des corrélations


| Mesure | Valeurs possibles | Interprétation du signe | Interprétation de l’amplitude | Quand l’utiliser ? |
|--------|-------------------|-------------------------|-------------------------------|---------------------|
| **Covariance** | -∞ → +∞ | Cov > 0 : X↑ → Y↑ (relation positive)  <br> Cov < 0 : X↑ → Y↓ (relation négative) <br> Cov ≈ 0 : pas de relation linéaire détectée | Impossible à comparer entre variables car dépend des unités | Pour savoir si les deux variables évoluent dans le même sens (mais pas pour mesurer l'intensité) |
| **Corrélation de Pearson** | -1 → +1 | r > 0 : relation linéaire positive <br> r < 0 : relation linéaire négative <br> r = 0 : pas de relation linéaire | |r| proche de 1 : relation forte <br> |r| ≈ 0.5 : relation modérée <br> |r| < 0.2 : relation très faible | Lorsque la relation est **linéaire** et que les variables sont relativement symétriques ou normales |
| **Corrélation de Spearman (ρ)** | -1 → +1 | Signe interprété comme Pearson | |ρ| proche de 1 : forte relation **monotone** | Lorsque la relation n’est pas linéaire mais **monotone** (croissante ou décroissante), ou en présence d’outliers |
| **Corrélation de Kendall (τ)** | -1 → +1 | Signe interprété comme Pearson | |τ| < ρ < r en général. <br> τ proche de 1 = forte concordance des rangs | Lorsque l’échantillon est **petit**, ou lorsque l’on veut une mesure robuste de concordance |



##  Comment interpréter la force d’une corrélation ?

| Valeur absolue | Interprétation |
|----------------|----------------|
| **0.00 – 0.19** | Très faible ou nulle |
| **0.20 – 0.39** | Faible |
| **0.40 – 0.59** | Modérée |
| **0.60 – 0.79** | Forte |
| **0.80 – 1.00** | Très forte |



##  À retenir

- La **covariance** indique seulement si X et Y varient **dans le même sens**.  
- Les **corrélations** normalisent la covariance et deviennent **comparables** et **interprétables**.  
- **Pearson** = linéaire, **Spearman** = monotone, **Kendall** = robuste.






#  CAS 2 — QUALI ↔ QUANTI
##Deux modalités (ex : sexe → montant_panier)



###  Tableau synthétique

| Élément | Contenu |
|--------|---------|
| Variables | sexe, montant_panier |
| Indicateurs | Moyennes par catégorie |
| Graphiques | Boxplot, violin plot |
| Objectif | Comparer deux groupes |


In [ ]:
df.groupby("sexe")["montant_panier"].agg(
    moyenne = "mean",
    mediane = "median",
    variance = "var",
    ecart_type = "std",
    minimum = "min",
    Q1 = lambda x: x.quantile(0.25),
    Q2 = "median",
    Q3 = lambda x: x.quantile(0.75),
    maximum = "max",
    IQR = lambda x: x.quantile(0.75) - x.quantile(0.25),
    etendue = lambda x: x.max() - x.min()
)

In [ ]:

plt.figure()
if sns:
    sns.boxplot(x="sexe", y="montant_panier", data=df)
else:
    groups = [g["montant_panier"].values for _, g in df.groupby("sexe")]
    labels = df["sexe"].unique()
    plt.boxplot(groups, labels=labels)
plt.title("Boxplot — montant du panier par sexe")
plt.show()


In [ ]:

if sns:
    plt.figure()
    sns.violinplot(x="sexe", y="montant_panier", data=df)
    plt.title("Violin plot — panier par sexe")
    plt.show()



##  Trois modalités ou plus (ex : segment_age → montant_panier)





| Élément | Contenu |
|--------|---------|
| Variables | segment_age, montant_panier |
| Indicateurs | Moyennes par groupe |
| Graphiques | Boxplot, barplot |
| Objectif | Comparer plusieurs groupes |


In [ ]:
df.groupby("segment_age")["montant_panier"].agg(
    moyenne = "mean",
    mediane = "median",
    variance = "var",
    ecart_type = "std",
    minimum = "min",
    Q1 = lambda x: x.quantile(0.25),
    Q2 = "median",
    Q3 = lambda x: x.quantile(0.75),
    maximum = "max",
    IQR = lambda x: x.quantile(0.75) - x.quantile(0.25),
    etendue = lambda x: x.max() - x.min()
)



In [ ]:

plt.figure()
if sns:
    sns.boxplot(x="segment_age", y="montant_panier", data=df)
plt.title("Boxplot — panier par segment d'âge")
plt.show()


In [ ]:

means = df.groupby("segment_age")["montant_panier"].mean()
plt.figure()
means.plot(kind="bar")
plt.ylabel("Montant du panier moyen")
plt.title("Barplot — panier moyen par segment d'âge")
plt.show()




# CAS 3 — QUALI ↔ QUALI  
**Exemple : sexe ↔ retour_produit**



###  Tableau synthétique

| Élément | Contenu |
|--------|---------|
| Variables | sexe, retour_produit |
| Indicateurs | Table de contingence |
| Graphiques | Countplot, heatmap |
| Objectif | Examiner l'association entre deux catégories |


In [ ]:

table = pd.crosstab(df["sexe"], df["retour_produit"])
table


In [ ]:

if sns:
    plt.figure()
    sns.countplot(x="sexe", hue="retour_produit", data=df)
    plt.title("Countplot — retour produit par sexe")
    plt.show()

    plt.figure()
    sns.heatmap(table, annot=True, fmt="d", cmap="Blues")
    plt.title("Heatmap — table de contingence")
    plt.show()


In [ ]:
# Table de contingence brute
table = pd.crosstab(df["sexe"], df["retour_produit"])

# Table des proportions (chaque ligne = 100%)
table_prop = table.div(table.sum(axis=1), axis=0) * 100

table_prop

In [ ]:
table_prop.plot(kind="bar", stacked=True, colormap="Blues")
plt.title("Répartition (%) du retour produit par sexe")
plt.ylabel("Pourcentage")
plt.show()


In [ ]:
plt.figure()
sns.heatmap(table_prop, annot=True, fmt=".1f", cmap="Blues")
plt.title("Heatmap — proportion de retours (%) par sexe")
plt.ylabel("Sexe")
plt.xlabel("Retour produit")
plt.show()



#  Récapitulatif 

- **QUANTI ↔ QUANTI** : covariance, corrélation, scatter plot  
- **QUALI ↔ QUANTI (2 modalities)** : moyennes par groupe, boxplot, violin plot  
- **QUALI ↔ QUANTI (3+ modalities)** : moyennes par groupe, boxplot, barplot  
- **QUALI ↔ QUALI** : table de contingence, countplot, heatmap  


